In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import logging
import os
import datetime

In [2]:
# # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)

In [3]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [4]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "directory_upgrade": True,
         "profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs", prefs)

In [5]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [6]:
# main page
main_page_url = "http://shopee.sg"
browser.get(main_page_url)

# remove ads, refresh again
for i in range(5):
    browser.get(main_page_url)
    time.sleep(10)

In [7]:
# login
LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
'''
有关Selenium Compound class names not permitted的错误
可以参考https://stackoverflow.com/questions/37771604/selenium-compound-class-names-not-permitted
使用css_selector解决即可
'''
# 点击登入，弹出账户密码输入框
LoginElem[1].click()
time.sleep(10)

In [8]:
acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
try:
    acc_password_input_elem[0].click()
    acc_password_input_elem[0].send_keys('tengus1.sg')
    acc_password_input_elem[1].click()
    acc_password_input_elem[1].send_keys('tengus1803')
except:
    time.sleep(30)
    acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
    acc_password_input_elem[0].click()
    acc_password_input_elem[0].send_keys('tengus1.sg')
    acc_password_input_elem[1].click()
    acc_password_input_elem[1].send_keys('tengus1803')
time.sleep(10)

In [9]:
Login_button_elem = browser.find_elements_by_css_selector('._2DvX7K._3j9-lD._3ddytl.SjORHu')
time.sleep(10)
Login_button_elem[0].click()
time.sleep(10)

In [10]:
# change current tab size
# browser.set_window_size(400, 862)    

In [11]:
# add fans from Top Seller's shop
top_shop_id = 11918
top_shop_url = 'https://shopee.sg/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
browser.get(top_shop_url)

In [12]:
# add certain number of fans
to_add_num_of_following = 400
num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))

In [13]:
# page down until we get at least 400 fans to cancel
# while num_of_following_display <= 1000:
    # body = browser.find_element_by_css_selector('body')
    # body.send_keys(Keys.PAGE_DOWN)
    # body.send_keys(Keys.END)
    # time.sleep(5)
    # num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
# num_of_following_display

In [14]:
following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
# len(following_buttons)

In [15]:
i = 0
total_add_num_of_following = to_add_num_of_following

In [16]:
browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)

In [17]:
while i <= total_add_num_of_following - 1:  
    while to_add_num_of_following > 0:
        if i + 1 > len(following_buttons):
            browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(10)
        
        following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
        logging.info('Now we have ' + str(len(following_buttons)) + ' following buttons on the screen.')
        logging.info('Now run the ' + str(i + 1) + ' time.')
        # if following_buttons[i].text == '+ 關注':
        # if following_buttons[i].text == '+ ติดตาม':
        if following_buttons[i].text == '+ Follow':
            # print(following_buttons[i].text)
            shopid = following_buttons[i].get_attribute('shopid')
            following_buttons[i].click()
            time.sleep(5)
            to_add_num_of_following -= 1
            logging.info(str(shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
        else:
            logging.warning('Skip this one. It\'s following already.')
        i += 1

 2019-10-27 20:12:42,422 - INFO - Now we have 20 following buttons on the screen.
 2019-10-27 20:12:42,424 - INFO - Now run the 1 time.
 2019-10-27 20:12:47,506 - INFO - 190597186 is following now, 399 following remains.
 2019-10-27 20:12:47,516 - INFO - Now we have 20 following buttons on the screen.
 2019-10-27 20:12:47,516 - INFO - Now run the 2 time.
 2019-10-27 20:12:52,591 - INFO - 187363565 is following now, 398 following remains.
 2019-10-27 20:12:52,602 - INFO - Now we have 20 following buttons on the screen.
 2019-10-27 20:12:52,602 - INFO - Now run the 3 time.
 2019-10-27 20:12:57,675 - INFO - 37782185 is following now, 397 following remains.
 2019-10-27 20:12:57,689 - INFO - Now we have 20 following buttons on the screen.
 2019-10-27 20:12:57,690 - INFO - Now run the 4 time.
 2019-10-27 20:13:02,763 - INFO - 122865879 is following now, 396 following remains.
 2019-10-27 20:13:02,775 - INFO - Now we have 20 following buttons on the screen.
 2019-10-27 20:13:02,776 - INFO - N

 2019-10-27 20:16:00,757 - INFO - Now run the 38 time.
 2019-10-27 20:16:05,842 - INFO - 859244 is following now, 362 following remains.
 2019-10-27 20:16:05,854 - INFO - Now we have 40 following buttons on the screen.
 2019-10-27 20:16:05,855 - INFO - Now run the 39 time.
 2019-10-27 20:16:10,938 - INFO - 192113055 is following now, 361 following remains.
 2019-10-27 20:16:10,950 - INFO - Now we have 40 following buttons on the screen.
 2019-10-27 20:16:10,951 - INFO - Now run the 40 time.
 2019-10-27 20:16:16,026 - INFO - 1975948 is following now, 360 following remains.
 2019-10-27 20:16:26,089 - INFO - Now we have 60 following buttons on the screen.
 2019-10-27 20:16:26,090 - INFO - Now run the 41 time.
 2019-10-27 20:16:31,163 - INFO - 183883 is following now, 359 following remains.
 2019-10-27 20:16:31,173 - INFO - Now we have 60 following buttons on the screen.
 2019-10-27 20:16:31,174 - INFO - Now run the 42 time.
 2019-10-27 20:16:36,255 - INFO - 3823411 is following now, 358 f

 2019-10-27 20:19:34,395 - INFO - 33464635 is following now, 325 following remains.
 2019-10-27 20:19:34,406 - INFO - Now we have 80 following buttons on the screen.
 2019-10-27 20:19:34,407 - INFO - Now run the 76 time.
 2019-10-27 20:19:39,487 - INFO - 191956025 is following now, 324 following remains.
 2019-10-27 20:19:39,497 - INFO - Now we have 80 following buttons on the screen.
 2019-10-27 20:19:39,498 - INFO - Now run the 77 time.
 2019-10-27 20:19:44,574 - INFO - 4882841 is following now, 323 following remains.
 2019-10-27 20:19:44,586 - INFO - Now we have 80 following buttons on the screen.
 2019-10-27 20:19:44,587 - INFO - Now run the 78 time.
 2019-10-27 20:19:49,662 - INFO - 26657978 is following now, 322 following remains.
 2019-10-27 20:19:49,674 - INFO - Now we have 80 following buttons on the screen.
 2019-10-27 20:19:49,674 - INFO - Now run the 79 time.
 2019-10-27 20:19:54,752 - INFO - 1361363 is following now, 321 following remains.
 2019-10-27 20:19:54,763 - INFO -

 2019-10-27 20:23:02,742 - INFO - 81378167 is following now, 288 following remains.
 2019-10-27 20:23:02,754 - INFO - Now we have 120 following buttons on the screen.
 2019-10-27 20:23:02,755 - INFO - Now run the 113 time.
 2019-10-27 20:23:07,832 - INFO - 46069016 is following now, 287 following remains.
 2019-10-27 20:23:07,845 - INFO - Now we have 120 following buttons on the screen.
 2019-10-27 20:23:07,846 - INFO - Now run the 114 time.
 2019-10-27 20:23:12,920 - INFO - 50858187 is following now, 286 following remains.
 2019-10-27 20:23:12,934 - INFO - Now we have 120 following buttons on the screen.
 2019-10-27 20:23:12,935 - INFO - Now run the 115 time.
 2019-10-27 20:23:18,010 - INFO - 52360415 is following now, 285 following remains.
 2019-10-27 20:23:18,025 - INFO - Now we have 120 following buttons on the screen.
 2019-10-27 20:23:18,026 - INFO - Now run the 116 time.
 2019-10-27 20:23:23,097 - INFO - 99889833 is following now, 284 following remains.
 2019-10-27 20:23:23,109

 2019-10-27 20:26:26,082 - INFO - Now run the 149 time.
 2019-10-27 20:26:31,152 - INFO - 191780195 is following now, 251 following remains.
 2019-10-27 20:26:31,167 - INFO - Now we have 160 following buttons on the screen.
 2019-10-27 20:26:31,168 - INFO - Now run the 150 time.
 2019-10-27 20:26:36,245 - INFO - 191780011 is following now, 250 following remains.
 2019-10-27 20:26:36,261 - INFO - Now we have 160 following buttons on the screen.
 2019-10-27 20:26:36,262 - INFO - Now run the 151 time.
 2019-10-27 20:26:41,353 - INFO - 191779993 is following now, 249 following remains.
 2019-10-27 20:26:41,367 - INFO - Now we have 160 following buttons on the screen.
 2019-10-27 20:26:41,368 - INFO - Now run the 152 time.
 2019-10-27 20:26:46,443 - INFO - 191779798 is following now, 248 following remains.
 2019-10-27 20:26:46,456 - INFO - Now we have 160 following buttons on the screen.
 2019-10-27 20:26:46,457 - INFO - Now run the 153 time.
 2019-10-27 20:26:51,537 - INFO - 191779408 is f

...

 2019-10-27 20:33:23,440 - INFO - 191756218 is following now, 178 following remains.
 2019-10-27 20:33:23,456 - INFO - Now we have 237 following buttons on the screen.
 2019-10-27 20:33:23,457 - INFO - Now run the 223 time.
 2019-10-27 20:33:28,565 - INFO - 191756116 is following now, 177 following remains.
 2019-10-27 20:33:28,586 - INFO - Now we have 237 following buttons on the screen.
 2019-10-27 20:33:28,588 - INFO - Now run the 224 time.
 2019-10-27 20:33:33,663 - INFO - 191756102 is following now, 176 following remains.
 2019-10-27 20:33:33,682 - INFO - Now we have 237 following buttons on the screen.
 2019-10-27 20:33:33,682 - INFO - Now run the 225 time.
 2019-10-27 20:33:38,753 - INFO - 191756023 is following now, 175 following remains.
 2019-10-27 20:33:38,770 - INFO - Now we have 237 following buttons on the screen.
 2019-10-27 20:33:38,771 - INFO - Now run the 226 time.
 2019-10-27 20:33:43,844 - INFO - 191755901 is following now, 174 following remains.
 2019-10-27 20:33:4

 2019-10-27 20:36:47,303 - INFO - Now run the 259 time.
 2019-10-27 20:36:52,377 - INFO - 191748565 is following now, 141 following remains.
 2019-10-27 20:36:52,397 - INFO - Now we have 277 following buttons on the screen.
 2019-10-27 20:36:52,398 - INFO - Now run the 260 time.
 2019-10-27 20:36:57,483 - INFO - 191748484 is following now, 140 following remains.
 2019-10-27 20:36:57,504 - INFO - Now we have 277 following buttons on the screen.
 2019-10-27 20:36:57,504 - INFO - Now run the 261 time.
 2019-10-27 20:37:02,577 - INFO - 191748454 is following now, 139 following remains.
 2019-10-27 20:37:02,597 - INFO - Now we have 277 following buttons on the screen.
 2019-10-27 20:37:02,597 - INFO - Now run the 262 time.
 2019-10-27 20:37:07,667 - INFO - 191748375 is following now, 138 following remains.
 2019-10-27 20:37:07,689 - INFO - Now we have 277 following buttons on the screen.
 2019-10-27 20:37:07,690 - INFO - Now run the 263 time.
 2019-10-27 20:37:12,775 - INFO - 191748329 is f

 2019-10-27 20:40:06,123 - INFO - Now we have 297 following buttons on the screen.
 2019-10-27 20:40:06,125 - INFO - Now run the 296 time.
 2019-10-27 20:40:11,194 - INFO - 49250592 is following now, 104 following remains.
 2019-10-27 20:40:11,214 - INFO - Now we have 297 following buttons on the screen.
 2019-10-27 20:40:11,215 - INFO - Now run the 297 time.
 2019-10-27 20:40:16,297 - INFO - 191741982 is following now, 103 following remains.
 2019-10-27 20:40:26,404 - INFO - Now we have 317 following buttons on the screen.
 2019-10-27 20:40:26,405 - INFO - Now run the 298 time.
 2019-10-27 20:40:31,478 - INFO - 191740368 is following now, 102 following remains.
 2019-10-27 20:40:31,500 - INFO - Now we have 317 following buttons on the screen.
 2019-10-27 20:40:31,502 - INFO - Now run the 299 time.
 2019-10-27 20:40:36,572 - INFO - 191740154 is following now, 101 following remains.
 2019-10-27 20:40:36,595 - INFO - Now we have 317 following buttons on the screen.
 2019-10-27 20:40:36,5

 2019-10-27 20:43:35,160 - INFO - Now we have 337 following buttons on the screen.
 2019-10-27 20:43:35,162 - INFO - Now run the 333 time.
 2019-10-27 20:43:40,239 - INFO - 191735826 is following now, 67 following remains.
 2019-10-27 20:43:40,262 - INFO - Now we have 337 following buttons on the screen.
 2019-10-27 20:43:40,263 - INFO - Now run the 334 time.
 2019-10-27 20:43:45,344 - INFO - 191735812 is following now, 66 following remains.
 2019-10-27 20:43:45,368 - INFO - Now we have 337 following buttons on the screen.
 2019-10-27 20:43:45,369 - INFO - Now run the 335 time.
 2019-10-27 20:43:50,449 - INFO - 191735763 is following now, 65 following remains.
 2019-10-27 20:43:50,472 - INFO - Now we have 337 following buttons on the screen.
 2019-10-27 20:43:50,473 - INFO - Now run the 336 time.
 2019-10-27 20:43:55,557 - INFO - 191735593 is following now, 64 following remains.
 2019-10-27 20:43:55,582 - INFO - Now we have 337 following buttons on the screen.
 2019-10-27 20:43:55,582 

 2019-10-27 20:47:04,357 - INFO - Now we have 377 following buttons on the screen.
 2019-10-27 20:47:04,358 - INFO - Now run the 370 time.
 2019-10-27 20:47:09,436 - INFO - 191724996 is following now, 30 following remains.
 2019-10-27 20:47:09,462 - INFO - Now we have 377 following buttons on the screen.
 2019-10-27 20:47:09,462 - INFO - Now run the 371 time.
 2019-10-27 20:47:14,541 - INFO - 191724970 is following now, 29 following remains.
 2019-10-27 20:47:14,566 - INFO - Now we have 377 following buttons on the screen.
 2019-10-27 20:47:14,567 - INFO - Now run the 372 time.
 2019-10-27 20:47:19,647 - INFO - 191724850 is following now, 28 following remains.
 2019-10-27 20:47:19,672 - INFO - Now we have 377 following buttons on the screen.
 2019-10-27 20:47:19,673 - INFO - Now run the 373 time.
 2019-10-27 20:47:24,754 - INFO - 191724806 is following now, 27 following remains.
 2019-10-27 20:47:24,781 - INFO - Now we have 377 following buttons on the screen.
 2019-10-27 20:47:24,781 

In [18]:
# 关闭
browser.quit()